# Problem 11.1
# Implemetation of Relif Algorithm for Feature Selection

In [1]:
import pandas as pd
import numpy as np

# 0. Pre-Processing

In [2]:
def dummies_fit(df):
    dummies_dict = {}
    for column in df.columns:
        if not np.issubdtype(df[column].dtype, np.number):
            dummies_dict[column] = {}
            i = 0
            for value in set(df[column].values):
                dummies_dict[column][value] = i
                i += 1
    return dummies_dict

In [3]:
def dummies_transfer(df, dum_dict):
    for column in df.columns:
        if column in dum_dict.keys():
            for key in dum_dict[column]:
                df.loc[df[column] == key, column] = dum_dict[column][key]

In [128]:
def min_max_fit(df, origin_dtypes):
    min_max_dict = {}
    for i in range(len(origin_dtypes)):
        if np.issubdtype(origin_dtypes[i], np.number):
            column = origin_dtypes.index[i]
            min_max_dict[column] = {}
            min_max_dict[column]['min'] = min(df[column].values)
            min_max_dict[column]['max'] = max(df[column].values)
    return min_max_dict

In [105]:
def min_max_transfer(df, min_max_dict):
    for column in df.columns:
        if column in min_max_dict.keys():
            for i in range(len(df)):
                df.loc[i, column] = (data.loc[i, column] - min_max_dict[column]['min']) / \
                                    (min_max_dict[column]['max'] - 
                                     min_max_dict[column]['min'])

# 1. Relief

In [166]:
def relief(data_df, target_name, origin_dtypes, tau):
    """
    This function returns the selected features of a relief algorithm 
    for a binary classification problem. 
    It has four inputs:
    1. data_df: the df of train data
    2. target_name: string, the column name of target
    3. origin_dtypes: the dtypes of each column
    4. tau: the threshold of values of selected features
    """
    
    x, y = data_df.drop([target_name], axis=1), data_df[target_name]
    features = list(x.columns)
    selected_features = [] # store the selected features into list
    
    # obtain the subset of different classes
    subset1, subset2 = x.iloc[y[y==0].index], x.iloc[y[y==1].index]
    pairs_dict = {} # store the information of the nearest rows of each row
    
    # compute the nearest row for each row
    for i in range(len(x)):
        pairs_dict[i] = {}
        row = x.iloc[i].values
        
        distance_intra, distance_inter = np.inf, np.inf

        # define the subset of the same class and different class for each row
        if y.iloc[i] == 0:
            intra_subset = subset1
            inter_subset = subset2
        else:
            intra_subset = subset2
            inter_subset = subset1
        # compute the distance of all other rows to the row i    
        for j in range(len(x)):
            # store the row index of the nearest row in the opposite class
            if j in inter_subset.index:
                d_inter = np.sum(row - x.iloc[j].values)**2
                
                if d_inter < distance_inter:
                    distance_inter = d_inter
                    pairs_dict[i]['inter'] = j
            # store the row index of the nearest row within the same class    
            elif j != i:
                d_intra = np.sum((row - x.iloc[j].values)**2)
                if d_intra < distance_intra:
                    distance_intra = d_intra
                    pairs_dict[i]['intra'] = j
                
                
    # compute the delta value for each feature
    for i in range(len(features)):
        feature = features[i]
        delta_i = 0
        
        # for numeric features:
        if np.issubdtype(origin_dtypes[i], np.number):
            
            for n in range(len(x)): # for each row
                
                # get the info of nearest row
                right_neigh, wrong_neigb = pairs_dict[n]['intra'], pairs_dict[n]['inter']
                # update the delta value
                delta_i -= (x.iloc[n][feature] - x.iloc[right_neigh][feature])**2
                delta_i += (x.iloc[n][feature] - x.iloc[wrong_neigb][feature]) **2
        else: # for categorical features
            for n in range(len(x)): # for each row
                # get the info of nearest row
                right_neigh, wrong_neigb = pairs_dict[n]['intra'], pairs_dict[n]['inter']
                
                # update delta value
                if x.iloc[n][feature] != x.iloc[right_neigh][feature]:
                    delta_i -= 1
                if x.iloc[n][feature] != x.iloc[wrong_neigb][feature]:
                    delta_i += 1
        # store the feature name if its delta is larger than threshold
        if delta_i >= tau:
            selected_features.append(feature)
    
    return selected_features 


# 2. Load Data

In [153]:
data = pd.read_csv('../../data/data.txt').drop(['Id'], axis=1)

In [154]:
data.head()

,color,root,sound,stripes,umbilical,touch,density,sugar,quality
0,dark-green,roll-up,dull,clear,hollow,hard,0.697,0.460,good
1,pitch-dark,roll-up,dead,clear,hollow,hard,0.744,0.376,good
2,pitch-dark,roll-up,dull,clear,hollow,hard,0.634,0.264,good
3,dark-green,roll-up,dead,clear,hollow,hard,0.608,0.318,good
4,white,roll-up,dull,clear,hollow,hard,0.556,0.215,good


In [155]:
origin_dtypes = data.drop(['quality'],axis=1).dtypes

In [156]:
dummies_transfer(data, dummies_fit(data))

In [157]:
data.head()

,color,root,sound,stripes,umbilical,touch,density,sugar,quality
0,1,1,0,0,1,0,0.697,0.460,0
1,0,1,1,0,1,0,0.744,0.376,0
2,0,1,0,0,1,0,0.634,0.264,0
3,1,1,1,0,1,0,0.608,0.318,0
4,2,1,0,0,1,0,0.556,0.215,0


In [158]:
min_max_transfer(data, min_max_fit(data, origin_dtypes))

In [159]:
data.head()

,color,root,sound,stripes,umbilical,touch,density,sugar,quality
0,1,1,0,0,1,0,0.906188,1.000000,0
1,0,1,1,0,1,0,1.000000,0.799043,0
2,0,1,0,0,1,0,0.780439,0.531100,0
3,1,1,1,0,1,0,0.728543,0.660287,0
4,2,1,0,0,1,0,0.624750,0.413876,0


# 3. Train and Predict

In [169]:
relief(data, 'quality', origin_dtypes, 1.5)

['root', 'sound', 'stripes', 'umbilical', 'touch']